<a href="https://colab.research.google.com/github/sumit-2006/DyslexiaSupport/blob/main/bart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Check GPU model

True
Tesla T4


In [3]:
!pip install transformers datasets torch sentencepiece


In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained model and tokenizer
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


In [8]:
from datasets import load_dataset

# Load a dataset with incorrect and correct text pairs
dataset = load_dataset("jfleg")  # JFLEG is a dataset for grammar correction
print(dataset["validation"][0])


{'sentence': 'So I think we can not live if old people could not find siences and tecnologies and they did not developped . ', 'corrections': ['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']}


In [11]:
def tokenize_function(examples):
    inputs = tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)
    # Handle multiple corrections by taking the first one or joining them
    corrections = [c[0] if isinstance(c, list) else c for c in examples["corrections"]]  # Take first correction if it's a list
    # corrections = [' '.join(c) if isinstance(c, list) else c for c in examples["corrections"]]  # Alternatively, join corrections with space

    targets = tokenizer(corrections, padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # You can increase this for better performance
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    push_to_hub=False
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
from transformers import Trainer, DataCollatorForSeq2Seq

# Data collator handles padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["validation"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)




<ipython-input-20-6372186389ab>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
!wandb login



wandb: Currently logged in as: tanu702428 (tanu702428-shri-g-s-institute-of-technology-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.142814
2,No log,0.141608
3,0.038600,0.155716


TrainOutput(global_step=567, training_loss=0.036823960418633894, metrics={'train_runtime': 716.2821, 'train_samples_per_second': 3.162, 'train_steps_per_second': 0.792, 'total_flos': 613561490472960.0, 'train_loss': 0.036823960418633894, 'epoch': 3.0})

In [22]:
trainer.save_model("./bart_grammar_corrector")
tokenizer.save_pretrained("./bart_grammar_corrector")


('./bart_grammar_corrector/tokenizer_config.json',
 './bart_grammar_corrector/special_tokens_map.json',
 './bart_grammar_corrector/vocab.json',
 './bart_grammar_corrector/merges.txt',
 './bart_grammar_corrector/added_tokens.json')

In [32]:
def correct_sentence(text):
    # Ensure inputs are on the same device as the model
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(model.device)
    outputs = model.generate(inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example test
print(correct_sentence("Hee are the topper of classs."))

Hence , he is the topper of the class .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [35]:
from google.colab import drive

# Unmount first to reset connection if it's already mounted
try:
  drive.flush_and_unmount()
  print('Drive unmounted')
except ValueError:
  pass

# Mount the drive
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Drive unmounted


MessageError: Error: credential propagation was unsuccessful